<a href="https://colab.research.google.com/github/KJ22222/ESSA/blob/main/11_21_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#8 파이썬 추천 시스템 패키지 - Surprise

In [2]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 5.3 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633994 sha256=b2f5ddde80a9865d2dc9757294b1691d4c9fead5383470f6e8082af185198ab9
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [3]:
import surprise 

print(surprise.__version__)

1.1.1


##Surprise 를 이용한 추천 시스템 구축

- 추천 데이터를 학습,테스트용으로 분리한뒤 svd 행렬 분해를 통해 잠재 요인 협업 필터링을 수행

In [4]:
from surprise import SVD
from surprise import Dataset 
from surprise import accuracy 
from surprise.model_selection import train_test_split

In [7]:
data = Dataset.load_builtin('ml-100k') 
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

- surprise는 자체적으로 로우 레벨의 데이터를 칼럼 레벨의 데이터로 변경하므로 원본인 로우 레벨의 사용자-아이템 평점 데이터를 데이터 세트로 적용해야함
- svd로 잠재 요인 협업 필터링 수행

In [8]:
algo = SVD()
algo.fit(trainset) 

- 예측 수행
  - test()는 사용자 아이템 평점 데이터 세트 전체에 대해서 추천을 예측하는 메서드
  - predict()는 개별 사용자와 영화에 대한 추천 평점을 반환

In [9]:
predictions = algo.test( testset )
print('prediction type :',type(predictions), ' size:',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.473370255877449, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.6452934836364754, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.073615944542846, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.629453217767324, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.65963464894313, details={'was_impossible': False})]

->was_impossible': True이면 예측값을 생성할 수 없는 데이터라는 의미
- 객체명.uid 형식으로 속성에 접근 가능

In [10]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3] ]

[('120', '282', 3.473370255877449),
 ('882', '291', 3.6452934836364754),
 ('535', '507', 4.073615944542846)]

- predict 사용
  - 개별 사용자의 아이템에 대한 추천 평점을 예측
  - 인자로 사용자 아이디, 아이템 아이디를 입력

In [11]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함. 
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.35   {'was_impossible': False}


->est로 예측 평점 반환
- 데스트 데이터 세트를 이용해 추천 예측 평점과 실제 평점 차이 평가
  1. rmse
  2. mse

In [12]:
accuracy.rmse(predictions)

RMSE: 0.9491


0.9490763533874891

##Surprise 주요 모듈 소개
- 로딩 되는 데이터 파일에 칼럼명을 가지는 헤더 문자열이 있어서는 안돼

In [15]:
import pandas as pd

ratings = pd.read_csv('/content/ratings.csv')
# ratings_noh.csv 파일로 unload 시 index 와 header를 모두 제거한 새로운 파일 생성.  
ratings.to_csv('/content/ratings_noh.csv', index=False, header=False)

- Dataset 모듈의 load_from_file()을 사용하여 데이터 로드
- redear 로 데이터 파일의 파싱 포맷을 지정
  - 현재 칼럼이 사용자,아이템 아이디,평점, 타임 이렇게 4개,각 필드 칼럼명,칼럼 분리문자,최소 최대평점을 입력해서 객체를 생성해야함

In [17]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data=Dataset.load_from_file('/content/ratings_noh.csv',reader=reader)

- 무비렌즈 데이터 형식이 아닌 경우 redear클래스로 설정해야함

In [18]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 수행시마다 동일한 결과 도출을 위해 random_state 설정 
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습 후 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

- svd 행렬 분해 기법을 이용해 추천 예측
- 잠재 요인 크기 k 값을 나타내는 파라미터 n_factors로 설정한뒤 데이터 학습 시키고 예측 진행

In [21]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('/content/ratings.csv') 
reader = Reader(rating_scale=(0.5, 5.0))

# ratings DataFrame 에서 컬럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 합니다. 
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

## 베이스 라인 평점
- 개인의 성향을 반영해 아이템 평가에 편향성요소를 반영하여 평점을 부과하는 것이 베이스 라인 평점
- 전체 평균 평점+사용자 편향 점수+아이템 편향 점수 공식으로 계싼


##교차 검증(Cross Validation)과 하이퍼 파라미터 튜닝
- 사이킷런과 유사한 cross_validate()와 gridsearchcv 클래스를 제공
- 5개의 학습,검증 폴드 데이터 세트로 분리해 교차 검증을 수행하고 rmse,mae로 성능 평가 진행

In [22]:
from surprise.model_selection import cross_validate 

# Pandas DataFrame에서 Surprise Dataset으로 데이터 로딩 
ratings = pd.read_csv('/content/ratings.csv') # reading data in pandas df
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0) 
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True) 

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8725  0.8770  0.8722  0.8682  0.8811  0.8742  0.0044  
MAE (testset)     0.6699  0.6723  0.6711  0.6684  0.6775  0.6719  0.0031  
Fit time          5.18    6.02    5.14    5.11    5.14    5.32    0.35    
Test time         0.32    0.25    0.15    0.26    0.16    0.23    0.06    


{'test_rmse': array([0.87254052, 0.8769698 , 0.87224182, 0.8682407 , 0.88108782]),
 'test_mae': array([0.66994868, 0.67232548, 0.671132  , 0.66844127, 0.67753623]),
 'fit_time': (5.1802568435668945,
  6.016587734222412,
  5.137151718139648,
  5.109818458557129,
  5.142442226409912),
 'test_time': (0.3221778869628906,
  0.25252676010131836,
  0.14823436737060547,
  0.25621485710144043,
  0.16114091873168945)}

-> 폴드별 성능 평가 수치와 전체 폴드의 평균 성능 평가 수치를 함께 보여줌
- 사이킷런과 유사하게 교차 검증을 진행
- 알고리즘에 따라 다를 수 있지만 svd의 경우 주로 점진적 하강 방식의 반복 횟수를 정하는 것과 k 크기를 지정하는 파라미터 튜닝

##Surprise 를 이용한 개인화 영화 추천 시스템 구축

- 학습과 데이터 세트를 분리하지 않고 전체를 학습 데이터로 사용

In [23]:
# 아래 코드는 train_test_split( )으로 분리되지 않는 Dataset에 fit( )을 호출하여 오류를 발생합니다.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)

AttributeError: ignored

- 전체 데이터로 학습 하려면DatasetAutoFolds 사용

In [25]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds = DatasetAutoFolds(ratings_file='/content/ratings_noh.csv', reader=reader)

#전체 데이터를 학습데이터로 생성함. 
trainset = data_folds.build_full_trainset()

In [27]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

- 특정 사용자를 9로 지정하여 아직 평점을 매기지 않은 영화 42로 선정한 뒤 예측 평점 계산

In [28]:
# 영화에 대한 상세 속성 정보 DataFrame로딩
movies = pd.read_csv('/content/movies.csv')

# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인. 
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [30]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


- 예측 평점은 3.13
- 사용자가 평점 매기지 않은 영화 전체 추출하여 예측 평점 순으로 영화 구하기

In [31]:
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
    
    # 모든 영화들의 movieId를 리스트로 생성. 
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742
